In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlt
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os

data = pd.DataFrame()

directory = "D:/analytix/arpit/ML/12/"

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        df['filename'] = filename
        data = pd.concat([data, df], ignore_index=True)

data.columns

In [3]:
data=data[['filename','duration', ' protocol_type', ' service', ' flag', ' src_bytes',
       ' dst_bytes', ' land', ' wrong_fragment', ' urgent', ' hot',
       ' num_failed_logins', ' logged_in', ' num_compromised', ' root_shell',
       ' su_attempted', ' num_root', ' num_file_creations', ' num_shells',
       ' num_access_files', ' num_outbound_cmds', ' is_host_login',
       ' is_guest_login', ' count', ' srv_count', ' serror_rate',
       ' srv_error_rate', ' rerror_rate', ' srv_rerror_rate', ' same_srv_rate',
       ' diff_srv_rate', ' srv_diff_host_rate', ' dst_host_count',
       ' dst_host_srv_count', ' dst_host_same_srv_rate',
       ' dst_host_diff_srv_rate', ' dst_host_same_src_port_rate',
       ' dst_host_srv_diff_host_rate', ' dst_host_serror_rate',
       ' dst_host_srv_serror_rate', ' dst_host_rerror_rate',
       ' dst_host_srv_rerror_rate']]

In [4]:
data.columns = data.columns.str.replace(' ', '')

In [5]:
data['filename'].replace({
    'Data_of_Attack_Back.csv':'other',
    'Data_of_Attack_Back_BufferOverflow.csv':'BufferOverflow',
    'Data_of_Attack_Back_FTPWrite.csv':'FTPWrite',
    'Data_of_Attack_Back_GuessPassword.csv':'GuessPassword',
    'Data_of_Attack_Back_NMap.csv':'NMap',
    'Data_of_Attack_Back_Neptune.csv':'Neptune',
    'Data_of_Attack_Back_Normal.csv':'Normal',
    'Data_of_Attack_Back_PortSweep.csv':'PortSweep',
    'Data_of_Attack_Back_RootKit.csv':'Rootkit',
    'Data_of_Attack_Back_Satan.csv':'Satan',
    'Data_of_Attack_Back_Smurf.csv':'Smurf'
},inplace=True)

data.shape

In [ ]:
num_cols = len(data.columns)
num_rows = (num_cols - 1) // 4 + 1 
fig, axes = plt.subplots(nrows=num_rows, ncols=4, figsize=(16, 4*num_rows))
for i, col in enumerate(data.columns):
    sns.histplot(data[col], ax=axes[i//4][i%4],kde=True)
    axes[i//4][i%4].set_title(col)
plt.tight_layout()
plt.show()

In [6]:
for i in data.columns:
    if (data[i].nunique()==1):
        print(i)

land
num_outbound_cmds


In [7]:
data.drop(columns = {'land','num_outbound_cmds'},inplace=True)
data.drop(data[data['su_attempted'] == .2].index, inplace = True)

In [8]:
nominal =['protocol_type', 'service', 'flag']
binary =['logged_in','root_shell','su_attempted','is_host_login','is_guest_login']
numeric =['duration', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'num_compromised', 'num_root','num_file_creations', 'num_shells','num_access_files','count', 'srv_count', 'serror_rate', 'srv_error_rate',
       'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
       'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
       'dst_host_serror_rate', 'dst_host_srv_serror_rate',
       'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

In [ ]:
data.isnull().sum()

In [ ]:
data.dtypes

In [ ]:
data['filename'].unique()

In [10]:
data['binaryactivity']=np.where((data.filename.isin(['BufferOverflow', 'FTPWrite', 'GuessPassword', 'Neptune', 'NMap', 'PortSweep', 'Rootkit', 'Satan', 'Smurf'])==True), 1, 0 )

creating one hot encoding for above variable

In [9]:
for i in nominal:
    print(i,data[i].nunique())

protocol_type 3
service 67
flag 11


In [8]:
data=pd.get_dummies(data,prefix={'protocol_type':'protocol', 'service':'service', 'flag':'flag'},columns = nominal, drop_first=False)

In [ ]:
data2 = data.copy()

FOR BINARY MODELLING

In [ ]:
train=data[data['filename']!='data']
target = data[data['filename']=='data']
train.drop(columns='filename',inplace=True)
target.drop(columns={'filename','binaryactivity'},inplace=True)

In [ ]:
x=train.copy()
y=x.pop('binaryactivity')

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

CLASSIFICATION MODELLING

we will use 
Logistic Classification.
Naive Bayes.
K-Nearest Neighbors.
Decision Tree.
Support Vector Machines.

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

models=[('svm',svm.SVC(kernel='linear')),
        ('LogisticRegression',LogisticRegression()),
        ('GaussianNB',GaussianNB()),
        ('KNeighborsClassifier',KNeighborsClassifier()),
        ('DecisionTreeClassifier',DecisionTreeClassifier())]

from sklearn.metrics import roc_auc_score,accuracy_score


for name,model in (models):
        model.fit(x_train,y_train)
        roc = roc_auc_score(y_test,model.predict(x_test))
        print("Accuracy score : ", (accuracy_score(y_test,model.predict(x_test))))
        print(name+ " has roc_auc_score ") 
        print(roc)

dropped above parameter because of low cpu power!!!

In [ ]:
from sklearn.metrics import roc_auc_score,accuracy_score
model = DecisionTreeClassifier()
model.fit(x_train,y_train)
model.score(x_test,y_test)
roc = roc_auc_score(y_test,model.predict(x_test))
print("Accuracy score : ", (accuracy_score(y_test,model.predict(x_test))))
print(" has roc_auc_score ") 
print(roc)
predicted=model.predict(target)

In [ ]:
predicted

for multinomial classification

In [ ]:
data=data2.copy()

In [11]:
train=data[data['filename']!='data']
target = data[data['filename']=='data']
train.drop(columns='binaryactivity',inplace=True)
target.drop(columns={'filename','binaryactivity'},inplace=True)

C:\Users\karpi\AppData\Local\Temp\ipykernel_10000\1657396628.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(columns='binaryactivity',inplace=True)
C:\Users\karpi\AppData\Local\Temp\ipykernel_10000\1657396628.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target.drop(columns={'filename','binaryactivity'},inplace=True)


In [12]:
x=train.copy()
y=x.pop('filename')

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [18]:
from sklearn.metrics import roc_auc_score,accuracy_score
model = DecisionTreeClassifier()
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.9997795603453554

In [21]:
print("Accuracy score : ", (accuracy_score(y_test,model.predict(x_test))))
predicted=model.predict(target)

Accuracy score :  0.9997795603453554


In [22]:
predicted

array(['Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',
       'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Norm